In [1]:
from agent_nodes import g2i

import rclpy
from rclpy.node import Node
from teleop_msgs.srv import G2I
from teleop_msgs.msg import Intent
from teleop_msgs.msg import Scene as SceneRos
from teleop_msgs.msg import Gestures as GesturesRos
import numpy as np
from srcmodules.Scenes import Scene
from srcmodules.Gestures import Gestures
from srcmodules.Actions import Actions
from srcmodules.Objects import Object
from srcmodules.SceneFieldFeatures import SceneFieldFeatures
#import time

Invalid MIT-MAGIC-COOKIE-1 key

#  Behaviour tree example

## Setup

- Scene is published on `/tree/scene_in`
- Gesture vector is published on `/tree/gestures_in`
- G2I Service setup on `/g2i`
    - Needs to have correct object types, ...

BTree

- Create BTree with `bt = BTreeLib.BTreeHandler()`
- Create BTree predictions with `bt.predict()` -> Action sequence is generated

### Init Gestures to Intent Service

In [2]:
rclpy.init()

In [3]:
global g2i
g2i.init()
g2i.rosnode = g2i.G2IRosNode(inference_type='1to1', ignore_unfeasible=True)
Object.all_types = Otypes = ['cup', 'drawer', 'object']

### Init scene & gesture vector publishers

In [4]:
pub_scene = g2i.rosnode.create_publisher(SceneRos, "/tree/scene_in", 5)
pub_gestures = g2i.rosnode.create_publisher(GesturesRos, "/tree/gestures_in", 5)

### Create B Tree

In [5]:
from srcmodules import BTreeLib

In [6]:
btree = BTreeLib.BTreeHandler()

In [7]:
import threading
executor = rclpy.executors.MultiThreadedExecutor()
executor.add_node(btree.behaviour_tree.node)
executor.add_node(g2i.rosnode)
# Spin in a separate thread
executor_thread = threading.Thread(target=executor.spin, daemon=True)
executor_thread.start()

# Ex. 1 
- put to drawer, when drawer is closed

1. Generate Scene and Gesture vector
2. Publish Scene and gesture vector info and generate the robotic action sequence 

In [8]:
g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(11), dtype=float))
g.probabilities.data[4] = 1.0

s = Scene(init='cup,drawer,object', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.drawer.position, dtype=float)

In [9]:
pub_scene.publish(sr)
pub_gestures.publish(g)

In [10]:
p = btree.predict()

[ WARN] Scene2BB: gripper attached!
=s============= Scene info. shape: [4, 4, 4]
cup,	cup,	[0. 0. 0.], empty,	|| [cup] >>
drawer,	drawer,	[0. 1. 0.], closed, cont: [],	|| [drawer] >>
object,	object,	[0. 2. 0.],	|| [object] >>
Robot: [0. 0. 3.], opened, rotation: 0.0, attached: cup
User added in feature vector! User ID: 0, User Name: Jan
action_id:  put
intent put, drawer

{-} root [✓]
    --> Scene2BB [✓] -- updated: Status.SUCCESS
    --> Gestures2BB [✓] -- updated: Status.SUCCESS
    --> GenIntent [✓] -- intent put, drawer,  generated
    {-} seq_lvl1 [✓]
        [o] to is drawer? [✓]
            --> to_not_eq_drawer [✕]
            [o] drawer precond.? [✓]
                --> drawer_state_not_in_preconditions [✕] -- drawer opened?: False, target_action is open?: False, fixing conditions
                {-} fix_drawer_precond [✓]
                    [o] place first? [✓]
                        --> hand_empty [✕]
                        --> place_object [✓] -- [4 PlaceObject] ta=place

In [11]:
p

[context_based_gesture_operation.msg.Intent(target_action='place', target_object='', auxiliary_parameters=''),
 context_based_gesture_operation.msg.Intent(target_action='open', target_object='drawer', auxiliary_parameters=''),
 context_based_gesture_operation.msg.Intent(target_action='pick_up', target_object='cup', auxiliary_parameters=''),
 context_based_gesture_operation.msg.Intent(target_action='put', target_object='drawer', auxiliary_parameters='')]

# Ex. 2
- move left

In [14]:
g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(9), dtype=float))
g.probabilities.data[1] = 1.0

s = Scene(init='cup,drawer,object', random=False)
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.object.position, dtype=float)

pub_scene.publish(sr)
pub_gestures.publish(g)
#time.sleep(0.5)

In [15]:
p = btree.predict()
p

[ WARN] Scene2BB: gripper attached!
User added in feature vector! User ID: 0, User Name: Jan
action_id:  move_left
intent move_left, object

{-} root [✓]
    --> Scene2BB [✓] -- updated: Status.SUCCESS
    --> Gestures2BB [✓] -- updated: Status.SUCCESS
    --> GenIntent [✓] -- intent move_left, object,  generated
    {-} seq_lvl1 [✓]
        [o] to is drawer? [✓]
            --> to_not_eq_drawer [✓]
            [o] drawer precond.?
                --> drawer_state_not_in_preconditions
                {-} fix_drawer_precond
                    [o] place first?
                        --> hand_empty
                        --> place_object
                    --> toggle_drawer
        [o] holding precond.? [✓]
            --> holding_precondition [✓]
            --> pick_to
        --> execute_ta [✓] -- [1 Execute TA] Status.SUCCESS! move_left,object
        --> delete_intent [✓]



[context_based_gesture_operation.msg.Intent(target_action='move_left', target_object='object', auxiliary_parameters='')]

# Ex. 3
- put to drawer, drawer is opened

In [16]:
g = GesturesRos()
g.probabilities.data = list(np.array(np.zeros(9), dtype=float))
g.probabilities.data[4] = 1.0

s = Scene(init='cup,drawer,object', random=False)
s.drawer.open()
Actions.do(s,('pick_up','cup'), ignore_location=True)
sr = s.to_ros(SceneRos())
sr.focus_point = np.array(s.drawer.position, dtype=float)

pub_scene.publish(sr)
pub_gestures.publish(g)

In [17]:
p = btree.predict()
p

[ WARN] Scene2BB: gripper attached!
User added in feature vector! User ID: 0, User Name: Jan
action_id:  put
intent put, drawer

{-} root [✓]
    --> Scene2BB [✓] -- updated: Status.SUCCESS
    --> Gestures2BB [✓] -- updated: Status.SUCCESS
    --> GenIntent [✓] -- intent put, drawer,  generated
    {-} seq_lvl1 [✓]
        [o] to is drawer? [✓]
            --> to_not_eq_drawer [✕]
            [o] drawer precond.? [✓]
                --> drawer_state_not_in_preconditions [✓] -- drawer opened?: True, target_action is open?: False, not fixing conditions
                {-} fix_drawer_precond
                    [o] place first?
                        --> hand_empty
                        --> place_object
                    --> toggle_drawer
        [o] holding precond.? [✓]
            --> holding_precondition [✓]
            --> pick_to
        --> execute_ta [✓] -- [1 Execute TA] Status.SUCCESS! put,drawer
        --> delete_intent [✓]



[context_based_gesture_operation.msg.Intent(target_action='put', target_object='drawer', auxiliary_parameters='')]

# Ex. 4
- put to drawer, not holding anything, target drawer opened
    
- Unfeasible, no object held in gripper -> action discarded before when checking action validity 